Alachua County restaurant inspection analysis¶
This will take restaurant inspection data by the state of Florida and format it in a more reader-friendly way for publication in print and online. We'll filter for the most egregious current violations at restaurants in Alachua County.

In [1]:
import pandas as pd
import bs4
import datetime
import re
import lxml
from urllib.request import urlopen
from urllib.error import HTTPError

In [2]:
# Read in state summary report for District, filter for needed fields and assign headers
try:
    insp = pd.read_csv("ftp://dbprftp.state.fl.us/pub/llweb/5fdinspi.csv", 
                               usecols=[2,14,18,80,81])
except IOError:
    print("The file is not accessible.")
insp.columns = ["CountyName", "InspectDate", 
                "NumHighVio", "LicenseID", "VisitID"]

# filter for alachua county restaurants with high violations
insp[(insp.CountyName == 'Alachua') & (insp.NumHighVio > 0)]
len(insp)

# change date string to date object, sort most recent# chang 
insp['InspectDate'] = pd.to_datetime(insp['InspectDate'])
insp = insp.sort_values('InspectDate', ascending=False)

# prefer to let user set timedelta or just remove. 
today = pd.to_datetime('today')
startDay = datetime.date.today() - datetime.timedelta(days=14)
insp = insp[(insp['InspectDate'] > startDay) & (insp['InspectDate'] < today)]
len(insp)

# takes LicenseID and VisitID, passes it into the urls for detailed reports#
result = []
for index, rows in insp.iterrows():
    visitID = rows['VisitID']
    licID = rows['LicenseID']
    urls = "https://www.myfloridalicense.com/inspectionDetail.asp? \
        InspVisitID= %s &licid= %s" % (visitID, licID)
    urls = urls.replace(' ', '')
    result.append(urls)

urlList = result

In [5]:
# function to pull relevant info from scrape into a dictionary
def make_details_dict(url):
    html = urlopen(url)
    soup = bs4.BeautifulSoup(html.read(), 'lxml')
    details = soup.find_all('font', {'face': 'verdana'})[10:]
    result = []
    global detailsDict
    
    for detail in details:
        siteName = details[0].text
        licNum = details[2].text
        siteRank = details[4].text
        expDate = details[6].text
        primeStatus = details[8].text
        secStatus = details[10].text
        siteAddress = details[12].text
        inspectResult = details[20].text
        observed1 = details[34].get_text
# maybe while loop to run through oberservation details        
#        while True:
#            n = n + 2 
#            str = observed
#            if len(str) , 2:
#                break
                
#        observed2 = details[36].text
#        observed3 = details[38].text
#        observed4 = details[40].text
#        observed5 = details[42].text
#        observed6 = details[44].text
#        observed7 = details[46].text
#        observed8 = details[48].text
#        observed9 = details[50].text
#        observed10 = details[52].text

    detailsDict = {
        'Restaurant': siteName,
        'License': licNum,
        'Rank': siteRank,
        'Expires': expDate,
        'Primary': primeStatus,
        'Secondary': secStatus,
        'Address': siteAddress,
        'Result': inspectResult,
        'Observed1': observed1,
#        'Observed2': observed2,
#        'Observed3': observed3,
#        'Observed4': observed4,
#        'Observed5': observed5,
#        'Observed6': observed6,
#        'Observed7': observed7,
#        'Observed8': observed8,
#        'Observed9': observed9,
#        'Observed10': observed10
    }
    result.append(detailsDict)    
        
    return detailsDict

# iterate through urls calling function above
for url in urlList:
    make_details_dict(url)
    print(detailsDict['Restaurant', 'Address', 'Observed1'])

KeyError: ('Restaurant', 'Address', 'Observed1')

Now need to cleanup data and feed into database